# Real life data

## Preprocessing

In [1]:
%load_ext autoreload 
%autoreload 2

import re
import sys
import hashlib

import pandas as pd
import numpy as np
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper
import tasks.preprocessing

In [2]:
df = pd.read_pickle("../data/attitudes_embedded.csv")
df = df.loc[
    :,
    [
        "tweet",
        "racism",
        "annotatorAge",
        "annotatorRace",
        "annotatorGender",
    ],
]

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

Clear out NaNs from dataset

In [4]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()

(np.int64(41), np.int64(41), np.int64(39))

In [5]:
def process_age_list(x, bins):
    if not isinstance(x, (list, tuple)):
        return None
    if any(pd.isna(age) for age in x):
        return None
    try:
        int_ages = [int(age) for age in x]
        return pd.cut(int_ages, bins=bins, include_lowest=True)
    except Exception:
        return None


all_ages = [
    age
    for sublist in df["annotatorAge"]
    if isinstance(sublist, (list, tuple))
    for age in sublist
    if pd.notna(age)
]
all_ages = list(map(int, all_ages))
num_bins = 4
bin_edges = [0, 20, 40, 60, 80]

df["annotatorAge"] = df["annotatorAge"].apply(
    lambda x: process_age_list(x, bin_edges)
)
df.annotatorRace = df.annotatorRace.apply(lambda x: None if ("na" in x) else x)

df.annotatorGender = df.annotatorGender.apply(
    lambda x: None if ("na" in x) else x
)

df["random"] = df.racism.apply(
    lambda x: [np.random.randint(1, 10) for _ in range(len(x))]
)
df = df.dropna()

In [6]:
tasks.preprocessing.find_inconsistent_rows(
    df, ["annotatorAge", "annotatorRace", "racism", "annotatorGender"]
).sum()

np.int64(0)

## Results

We hypothesize that all of the comments belong in the same discussion.

In [7]:
df2 = df.copy()
df2["fake_index"] = 1

In [8]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorAge",
    value_col="racism",
    comment_key_col="tweet",
)

1          
                   value    pvalue
(20.0, 40.0]    0.034749  0.668376
(40.0, 60.0]   -0.057702  0.583333
(60.0, 80.0]   -0.272437  1.000000
(-0.001, 20.0]  0.000000  1.000000

In [9]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorRace",
    value_col="racism",
    comment_key_col="tweet",
)

/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dimits/conda/envs/apunim-experiments/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


1          
                  value    pvalue
black          0.520747  0.258562
white         -0.258368  0.750427
hisp           0.000000  1.000000
other          0.000000  1.000000
middleEastern  0.000000       NaN

In [10]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="annotatorGender",
    value_col="racism",
    comment_key_col="tweet",
)

1          
              value    pvalue
man        0.142529  0.517182
woman      0.049424  0.546595
nonBinary  0.000000  1.000000

In [11]:
tasks.run_helper.results(
    df2,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="racism",
    comment_key_col="tweet",
)

1          
      value    pvalue
3 -0.149784  0.784672
7  0.142915  0.739464
4 -0.082055  0.768966
9 -0.000810  0.773519
8 -0.090854  0.777372
2 -0.041601  0.728522
1  0.010420  0.727273
5 -0.072643  0.759398
6 -0.073155  0.711111